In [14]:
import matplotlib.pyplot as plt
from sklearn.tree import ExtraTreeRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
import duckdb
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score, make_scorer
import warnings
warnings.filterwarnings('ignore')

DB_PATH = 'numero.duckdb'

def get_data(limit=100):
    with duckdb.connect(DB_PATH) as con:
        table_name = con.execute("SHOW TABLES").fetchall()
        df = con.execute(f"SELECT * FROM {table_name[0][0]}").df()
    return df

def JSON_to_DF(df_json):
    df = df_json.copy()
    df['films'] = df['data'].dropna().apply(lambda x: json.loads(x)['films'])
    df = df.explode('films').reset_index(drop=True)
    json_expanded = pd.json_normalize(df['films'])
    json_expanded.columns = [f"data.films.{col}" for col in json_expanded.columns]
    df = df.drop(columns=['data', 'films']).join(json_expanded)
    df.columns = [col.replace("data.films.", "") for col in df.columns]
    return df

def prepare_concurrent_features(df):
    # Calculate first week gross for each film
    df['first_week_gross'] = df['week.gross'].fillna(0) + df['weekend.gross'].fillna(0)

    # Group by release date
    features = []
    for idx, row in df.iterrows():
        date = row['releaseDate'] if 'releaseDate' in row else row.get('week_date', None)
        censor = row['censorRating']
        dist = row['distributorName']
        # Get all other films released the same week (excluding this film)
        concurrent = df[(df['releaseDate'] == date) & (df.index != idx)] if 'releaseDate' in df else df[(df['week_date'] == date) & (df.index != idx)]
        concurrent_grosses = concurrent['first_week_gross'].values if len(concurrent) > 0 else [0]
        features.append({
            'week.theatreCount': row.get('week.theatreCount', 0),
            'week.screenCount': row.get('week.screenCount', 0),
            'weekend.theatreCount': row.get('weekend.theatreCount', 0),
            'weekend.screenCount': row.get('weekend.screenCount', 0),
            'concurrent_mean_gross': np.mean(concurrent_grosses),
            'concurrent_median_gross': np.median(concurrent_grosses),
            'censorRating': censor,
            'distributorName': dist,
            'first_week_gross': row['first_week_gross']
        })
    features_df = pd.DataFrame(features)
    return features_df

def prepare_features(df):
    features_df = prepare_concurrent_features(df)
    numeric_columns = ['week.theatreCount', 'week.screenCount',
                       'weekend.theatreCount', 'weekend.screenCount',
                       'concurrent_mean_gross', 'concurrent_median_gross'
    ]
    X = features_df[numeric_columns].copy()
    scaler = StandardScaler()
    X[numeric_columns] = scaler.fit_transform(X[numeric_columns])
    le_censor = LabelEncoder()
    le_dist = LabelEncoder()
    X['censorRating_enc'] = le_censor.fit_transform(features_df['censorRating'])
    X['distributorName_enc'] = le_dist.fit_transform(features_df['distributorName'])
    y = np.log1p(features_df['first_week_gross'])
    return X, y, scaler, le_censor, le_dist

def evaluate_predictions(y_true, y_pred, model_name):
    mape = mean_absolute_percentage_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"\n{model_name} Metrics:")
    print(f"MAPE: {mape:.2%}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R2 Score: {r2:.4f}")

df_json = get_data()
df = JSON_to_DF(df_json)
# Calculate total gross for each movie
df['total_gross'] = df['week.gross'].fillna(0) + df['weekend.gross'].fillna(0)

threshold = df['total_gross'].quantile(0.95)
df['category'] = df['total_gross'].apply(lambda x: 'Blockbuster' if x > threshold else 'Normal')
df = df[df['category'] == 'Normal']

X, y, scaler, le_censor, le_dist = prepare_features(df)
tscv = TimeSeriesSplit(n_splits=5)
param_grid = {
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'max_features': ['sqrt', 'log2']
}
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)
print("\n=== Running Grid Search for ExtraTreeRegressor (concurrent films model) ===")
grid_search = GridSearchCV(
    estimator=ExtraTreeRegressor(),
    param_grid=param_grid,
    cv=tscv,
    scoring=mape_scorer,
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X, y)
best_params = grid_search.best_params_
best_score = grid_search.best_score_
best_model = grid_search.best_estimator_
print(f"\nBest Parameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")
print(f"\nBest MAPE: {-best_score:.4f}")
# Evaluate on folds
for i, (train_idx, test_idx) in enumerate(tscv.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    model = ExtraTreeRegressor(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_test_orig = np.expm1(y_test)
    y_pred_orig = np.expm1(y_pred)
    evaluate_predictions(y_test_orig, y_pred_orig, f"ExtraTree - Fold {i+1}")
# # Example prediction for a new film
# new_film = {'censorRating': 'PG-13', 'distributorName': 'Universal Pictures'}
# concurrent_films = [{'gross': 100000}, {'gross': 130000}]
# # Aggregate concurrent features
# count = len(concurrent_films)
# total = sum(f['gross'] for f in concurrent_films)
# mean = np.mean([f['gross'] for f in concurrent_films]) if concurrent_films else 0
# median = np.median([f['gross'] for f in concurrent_films]) if concurrent_films else 0
# max_g = max([f['gross'] for f in concurrent_films]) if concurrent_films else 0
# X_new = np.array([[count, total, mean, median, max_g]])
# # X_new = scaler.transform(X_new)
# # censor_enc = le_censor.transform([new_film['censorRating']])[0] if new_film['censorRating'] in le_censor.classes_ else -1
# # dist_enc = le_dist.transform([new_film['distributorName']])[0] if new_film['distributorName'] in le_dist.classes_ else -1
# # X_pred = np.concatenate([X_new[0], [censor_enc, dist_enc]]).reshape(1, -1)
# # y_pred = best_model.predict(X_pred)
# # pred_gross = float(np.expm1(y_pred[0]))
# # print(f"Predicted first week gross for new film: ${pred_gross:,.2f}")




=== Running Grid Search for ExtraTreeRegressor (concurrent films model) ===
Fitting 5 folds for each of 8 candidates, totalling 40 fits

Best Parameters:
max_depth: 10
max_features: log2
min_samples_split: 2

Best MAPE: 0.0804

ExtraTree - Fold 1 Metrics:
MAPE: 243.64%
RMSE: 3069941.80
R2 Score: 0.6244

ExtraTree - Fold 2 Metrics:
MAPE: 194.00%
RMSE: 4386932.53
R2 Score: 0.5267

ExtraTree - Fold 3 Metrics:
MAPE: 219.89%
RMSE: 2878166.19
R2 Score: 0.6359

ExtraTree - Fold 4 Metrics:
MAPE: 177.55%
RMSE: 3645615.34
R2 Score: 0.5821

ExtraTree - Fold 5 Metrics:
MAPE: 207.16%
RMSE: 3010962.02
R2 Score: 0.6223
